# Image https://hub.docker.com/r/bitnami/elasticsearch#!&_escaped_fragment_=

Ports: 9200, 9201, 9300
Mount: -v /path/to/elasticsearch-data-persistence:/bitnami/elasticsearch/data

curl -XPUT -H "Content-Type: application/json" http://localhost:9200/_cluster/settings -d '{ "transient": { "cluster.routing.allocation.disk.threshold_enabled": false } }'

curl -XPUT -H "Content-Type: application/json" http://localhost:9200/_all/_settings -d '{"index.blocks.read_only_allow_delete": null}


http://localhost:9200/?pretty

es5 container

Array mapping: https://stackoverflow.com/questions/67581594/elasticsearch-mapping-for-array

Habr guide: https://habr.com/ru/articles/280488/'

All orders: http://localhost:9200/order/_search?pretty
All masters: http://localhost:9200/master/_search?pretty

In [94]:
import requests

In [95]:
r = requests.put('http://localhost:9200/_cluster/settings', json={ "transient": { "cluster.routing.allocation.disk.threshold_enabled": False } })

In [96]:
r.json()

{'acknowledged': True,
 'persistent': {},
 'transient': {'cluster': {'routing': {'allocation': {'disk': {'threshold_enabled': 'false'}}}}}}

In [97]:
r = requests.put('http://localhost:9200/_all/_settings', json={"index.blocks.read_only_allow_delete": None})

In [98]:
r.json()

{'acknowledged': True}

In [99]:
import json
from elasticsearch import Elasticsearch

In [242]:
client = Elasticsearch([{"host": "127.0.0.1", "scheme": "http", "port": 9200}])

client.ping()

False

# Index 1 - orders

In [167]:
indexName = "order"
client.indices.delete(index=indexName)
# client.indices.create(index=indexName)

ObjectApiResponse({'acknowledged': True})

In [168]:
analyzer_settings = {
  "settings": {
    "analysis": {
      "filter": {
        "ru_stop": {
          "type": "stop",
          "stopwords": "_russian_"
        },
        "ru_stemmer": {
          "type": "stemmer",
          "language": "russian"
        }
      },
      "analyzer": {
        "custom_analyzer": {
          "type": "custom",
          "tokenizer": "standard",
          "filter": [
            "lowercase",
            "ru_stop",
            "ru_stemmer"
          ]
        }
      }
    }
  }
}

In [169]:
mappings_order = {
  "mappings": {
    "properties": {
      "order_id": {
        "type": "keyword"
      },
      "order_date": {
        "type": "keyword"
        # "format": "yyyy-MM-dd"
      },
      "order_customer_id": {
        "type": "keyword"
      },
      "order_customer_desc": {
        "type": "text",
        "analyzer": "custom_analyzer"
      },
      "order_details_desc": {
        "type": "text",
        "analyzer": "custom_analyzer"
      },
      "order_due_date": {
        "type": "keyword"
      },
      "order_fact_completion_date": {
        "type": "keyword"
      },
      "order_parts": {
        "type": "text",
        "analyzer": "custom_analyzer",
        "fields": {
          "order_parts_row": {
            "type": "text",
            "analyzer": "custom_analyzer",
          }
        }
      },
      "repair_types": {
        "type": "text",
        "analyzer": "custom_analyzer",
        "fields": {
          "repair_types_row": {
            "type": "text",
            "analyzer": "custom_analyzer",
          }
        }
      },
      "order_price": {
        "type": "keyword"
      },
      "order_master_id": {
        "type": "keyword"
      }
    }
  }
}


client.indices.create(index=indexName, 
                            # doc_type="diseases", 
                           # include_type_name="true", 
                           body={**analyzer_settings, **mappings_master}
                          )

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'order'})

In [170]:
with open('../orders.json', 'r') as f:
    dataStore = json.load(f)

for data in dataStore:
    try:
        client.index(
            index=data["index"],
            # doc_type=data["doc_type"],
            id=data["id"],
            body=data["body"]
        )
    except Exception as e:
        print(e, end="")
print("index 'order' data inserted")

index 'order' data inserted


# Query 1 - orders

In [241]:
indexName = "order"
searchFrom = 0
searchSize = 5

# searchBody = {
#   "_source": True, #["master_desc"], # True - all data
#   "query": {
#     "match": {
#       "master_desc": "стаж"
#     }
#   },
#   "size": searchSize
# }

searchBody = {
  "size": searchSize,
  # "query" : {
  #   "match_all" : {}
  # }
  "_source": True,
  # "query": {},
  "aggs": {
    "over_months": {
      "date_histogram": {
        "field": "order_date",
        "calendar_interval": "month",
        "format": "yyyy-MM-dd" # control the output format
      },
        "aggs":{
            "over_parts": {
              "terms": {
                "field": "order_parts.keyword"
                # "fielddata": True
              },
            }
        }
    }
  }
}

result = client.search(index=indexName, body=searchBody)

ConnectionTimeout: Connection timed out

In [ ]:
# result

In [ ]:
result['aggregations']['over_months']['buckets']

In [221]:
# result['hits']

# Index 2 - masters

In [9]:
indexName = "master"
client.indices.delete(index=indexName)
# client.indices.create(index=indexName)

ObjectApiResponse({'acknowledged': True})

In [10]:
# analyzer_settings = {
#   "settings": {
#     "analysis": {
#       "filter": {
#         "ru_stop": {
#           "type": "stop",
#           "stopwords": "_russian_"
#         },
#         "ru_stemmer": {
#           "type": "stemmer",
#           "language": "russian"
#         }
#       },
#       "analyzer": {
#         "custom_analyzer": {
#           "type": "custom",
#           "tokenizer": "standard",
#           "filter": [
#             "lowercase",
#             "ru_stop",
#             "ru_stemmer"
#           ]
#         }
#       }
#     }
#   }
# }

In [11]:
mappings_master = {
  "mappings": {
    "properties": {
      "master_id": {
        "type": "keyword"
      },
      "master_desc": {
        "type": "text",
        "analyzer": "custom_analyzer"
      },
      "master_feedbacks": {
        "type": "text",
        "analyzer": "custom_analyzer",
        "fields": {
          "master_feedbacks_row": {
            "type": "text",
            "analyzer": "custom_analyzer",
          }
        }
      }
    }
  }
}


client.indices.create(index=indexName, 
                            # doc_type="diseases", 
                           # include_type_name="true", 
                           body={**analyzer_settings, **mappings_master}
                          )

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'master'})

In [12]:
with open('../masters.json', 'r') as f:
    dataStore = json.load(f)

for data in dataStore:
    try:
        client.index(
            index=data["index"],
            # doc_type=data["doc_type"],
            id=data["id"],
            body=data["body"]
        )
    except Exception as e:
        print(e, end="")
print("index 'master' data inserted")

ok


# Query 2 - masters

In [61]:
indexName = "master"
searchFrom = 0
searchSize = 10

# 2
# searchBody = {
#     "_source": ["name"],
#     "query": {
#         "simple_query_string": {
#             "query": 'стаж',
#             "fields": ["master_desc"]
#         }
#     },
#     "size": searchSize
# }

searchBody = {
  "_source": True, #["master_desc"], # True - all data
  "query": {
    "match": {
      "master_desc": "стаж"
    }
  },
  "size": searchSize
}

# searchBody = {}
result = client.search(index=indexName, body=searchBody)
# , body=searchBody, from_=1, size=5)

In [62]:
result

ObjectApiResponse({'took': 8, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 44, 'relation': 'eq'}, 'max_score': 0.7704413, 'hits': [{'_index': 'master', '_id': '247830', '_score': 0.7704413, '_source': {'master_id': 247830, 'master_desc': 'Лора Эдуардовна Быкова, Стаж Работы: 14 Л./Г..', 'master_feedbacks': ['медлительный, аккуратный.', 'аккуратный, ворчливый.', 'ворчливый, аккуратный.']}}, {'_index': 'master', '_id': '578414', '_score': 0.7704413, '_source': {'master_id': 578414, 'master_desc': 'Терентий Валентинович Бобылев, Стаж Работы: 10 Л./Г..', 'master_feedbacks': ['медлительный, ворчливый.', 'аккуратный, медлительный.', 'ворчливый, аккуратный.']}}, {'_index': 'master', '_id': '435930', '_score': 0.7704413, '_source': {'master_id': 435930, 'master_desc': 'Шарапова Раиса Владимировна, Стаж Работы: 8 Л./Г..', 'master_feedbacks': ['аккуратный, ворчливый.', 'ворчливый, аккуратный.']}}, {'_index': 'master', '_id':

In [63]:
result['hits']

{'total': {'value': 44, 'relation': 'eq'},
 'max_score': 0.7704413,
 'hits': [{'_index': 'master',
   '_id': '247830',
   '_score': 0.7704413,
   '_source': {'master_id': 247830,
    'master_desc': 'Лора Эдуардовна Быкова, Стаж Работы: 14 Л./Г..',
    'master_feedbacks': ['медлительный, аккуратный.',
     'аккуратный, ворчливый.',
     'ворчливый, аккуратный.']}},
  {'_index': 'master',
   '_id': '578414',
   '_score': 0.7704413,
   '_source': {'master_id': 578414,
    'master_desc': 'Терентий Валентинович Бобылев, Стаж Работы: 10 Л./Г..',
    'master_feedbacks': ['медлительный, ворчливый.',
     'аккуратный, медлительный.',
     'ворчливый, аккуратный.']}},
  {'_index': 'master',
   '_id': '435930',
   '_score': 0.7704413,
   '_source': {'master_id': 435930,
    'master_desc': 'Шарапова Раиса Владимировна, Стаж Работы: 8 Л./Г..',
    'master_feedbacks': ['аккуратный, ворчливый.', 'ворчливый, аккуратный.']}},
  {'_index': 'master',
   '_id': '111767',
   '_score': 0.7704413,
   '_sourc